# Project 3: Fraud Detection

In this third project, the main objective is to develop a fraud detection model capable of predicting fraudulent bank transactions. The implementation places particular emphasis on the following key areas:
- Unsupervised learning
- Semi-supervised learning
- Dealing with class imbalance
- Using advanced performance metrics to evaluate detection models under different inspection effort thresholds.

In [1]:
# Includes
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
# Load data
df = pd.read_csv("data/sales.csv")
df = df.set_index('Unnamed: 0') # set index
df.index.name = None
df

,ID,Prod,Quant,Val,Insp
1,v1,p1,182.0,1665.0,unkn
2,v2,p1,3072.0,8780.0,unkn
3,v3,p1,20393.0,76990.0,unkn
4,v4,p1,112.0,1100.0,unkn
5,v3,p1,6164.0,20260.0,unkn
...,...,...,...,...,...
411814,v5918,p4126,113.0,9330.0,unkn
411815,v2540,p4126,118.0,3355.0,unkn
411816,v5896,p4127,143.0,1835.0,unkn
411817,v1796,p4127,102.0,1445.0,unkn


In [22]:
df["Unitprice"] = df["Val"] / df["Quant"] 
df

,ID,Prod,Quant,Val,Insp,Unitprice
1,v1,p1,182.0,1665.0,unkn,9.148352
2,v2,p1,3072.0,8780.0,unkn,2.858073
3,v3,p1,20393.0,76990.0,unkn,3.775315
4,v4,p1,112.0,1100.0,unkn,9.821429
5,v3,p1,6164.0,20260.0,unkn,3.286827
...,...,...,...,...,...,...
411814,v5918,p4126,113.0,9330.0,unkn,82.566372
411815,v2540,p4126,118.0,3355.0,unkn,28.432203
411816,v5896,p4127,143.0,1835.0,unkn,12.832168
411817,v1796,p4127,102.0,1445.0,unkn,14.166667
